<a href="https://colab.research.google.com/github/andreidore/aicrowd_practice/blob/master/tmpmn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [2]:
!rm -rf data
!mkdir data
!wget https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/tmpmn/v0.1/test.csv
!wget https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/tmpmn/v0.1/train.csv
!mv train.csv data/train.csv
!mv test.csv data/test.csv

--2020-05-20 16:52:26--  https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/tmpmn/v0.1/test.csv
Resolving s3.eu-central-1.wasabisys.com (s3.eu-central-1.wasabisys.com)... 130.117.252.10, 130.117.252.13, 130.117.252.16, ...
Connecting to s3.eu-central-1.wasabisys.com (s3.eu-central-1.wasabisys.com)|130.117.252.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13261 (13K) [text/csv]
Saving to: ‘test.csv’

test.csv            100%[===================>]  12.95K  56.5KB/s    in 0.2s    

2020-05-20 16:52:27 (56.5 KB/s) - ‘test.csv’ saved [13261/13261]

--2020-05-20 16:52:28--  https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/tmpmn/v0.1/train.csv
Resolving s3.eu-central-1.wasabisys.com (s3.eu-central-1.wasabisys.com)... 130.117.252.10, 130.117.252.13, 130.117.252.16, ...
Connecting to s3.eu-central-1.wasabisys.com (s3.eu-central-1.wasabisys.com)|130.117.252.10|:443... connected.
HTTP request sent, awaiting response... 2

In [0]:
all_data_path = "data/train.csv" #path where data is stored

In [7]:
all_data = pd.read_csv(all_data_path)
all_data.head()

,Max_temperature,Min_temperature,Dewpoint,Precipitation,Sea_level_pressure,Standard_pressure,Visibility,Wind_speed,Max_wind_speed,Mean_temperature
0,86.5,57.6,56.5,0.0,29.93,7.4,7.48,13.8,34.28,72.4
1,55.6,37.4,36.1,0.0,30.30,7.5,12.70,20.8,34.28,46.6
2,85.6,62.4,52.8,0.0,29.94,7.4,10.40,16.1,34.28,74.3
3,75.2,53.6,46.9,0.0,29.93,7.3,19.70,25.3,34.28,62.8
4,60.8,34.0,41.9,0.0,30.04,6.4,9.09,16.1,34.28,49.9


In [0]:
X_train, X_val= train_test_split(all_data, test_size=0.2, random_state=42)

In [0]:
X_train,y_train = X_train.iloc[:,:-1],X_train.iloc[:,-1]
X_val,y_val = X_val.iloc[:,:-1],X_val.iloc[:,-1]

In [0]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_val)

In [13]:
mse = mean_squared_error(y_val,y_pred)
mae = mean_absolute_error(y_val,y_pred)

print("MSE of the model is :" ,mse)
print("MAE of the model is :" ,mae)

MSE of the model is : 1.7495436632693695
MAE of the model is : 0.9798772295819201


In [0]:
final_test_path = "data/test.csv"
final_test = pd.read_csv(final_test_path)

submission = regressor.predict(final_test)


submission = pd.DataFrame(submission)
submission.to_csv('submission.csv',header=['mean_temp'],index=False)

In [0]:
from google.colab import files
files.download('submission.csv') 